In [ ]:
#@title Migrate ligands
#@markdown This notebook automates the migration of ligands from a homologue
#@markdown (provided or determined via blast) to the provided model,
#@markdown using PyRosetta. This includes:

#@markdown * parameterisation of unusual residues
#@markdown * poses are superimposed via neighbouring residues of the ones of ligands of interest
#@markdown (the equivalence of the neighbouring residues is determined by sequence aligment)
#@markdown * relaxation of the merge, with constraints based on hydrogen bonds form the donor pose

#@markdown This is for small molecules only. Not for NCAA or nucleic acids.

#@markdown The search for homologues does not use PyRosetta so can be stripped down.

#@markdown This notebook from [github.com/matteoferla/pyrosetta_help](https://github.com/matteoferla/pyrosetta_help).

#@markdown It can be opened in Colabs via [https://colab.research.google.com/github/matteoferla/pyrosetta_help/blob/main/colabs/colabs-pyrosetta-migrate_ligands.ipynb](https://colab.research.google.com/github/matteoferla/pyrosetta_help/blob/main/colabs/colabs-pyrosetta-migrate_ligands.ipynb)



In [ ]:
#@title Installation
#@markdown Installing PyRosetta with optional backup to your drive (way quicker next time!).
#@markdown Note that PyRosetta occupies some 10 GB, so you'll need to be on the 100 GB plan of Google Drive (it's one pound a month).

#@markdown The following is not the real password. However, the format is similar.
username = 'boltzmann' #@param {type:"string"}
password = 'constant' #@param {type:"string"}
#@markdown Release to install:
_release = 'release-295' #@param {type:"string"}
#@markdown Use Google Drive for PyRosetta (way faster next time, but takes up space)
#@markdown (NB. You may be prompted to follow a link and possibly authenticate and then copy a code into a box
use_drive = True #@param {type:"boolean"}
#@markdown Installing `rdkit` and `rdkit_to_params` allows the creation of custom topologies (params) for new ligands
#@markdown **Code to use these not copy-pasted over**
install_rdkit = True #@param {type:"boolean"}


import sys
import platform
import os
assert platform.dist()[0] == 'Ubuntu'
py_version = str(sys.version_info.major) + str(sys.version_info.minor)
if use_drive:
  from google.colab import drive
  drive.mount('/content/drive')
  _path = '/content/drive/MyDrive'
  os.chdir(_path)
else:
  _path = '/content'
if not any(['PyRosetta4.Release' in filename for filename in os.listdir()]):
  assert not os.system(f'curl -u {username}:{password} https://graylab.jhu.edu/download/PyRosetta4/archive/release/PyRosetta4.Release.python{py_version}.ubuntu/PyRosetta4.Release.python{py_version}.ubuntu.{_release}.tar.bz2 -o /content/a.tar.bz2')
  assert not os.system('tar -xf /content/a.tar.bz2')
assert not os.system(f'pip3 install -e {_path}/PyRosetta4.Release.python{py_version}.ubuntu.{_release}/setup/')
assert not os.system(f'pip3 install pyrosetta-help biopython')
if install_rdkit:
  assert not os.system(f'pip3 install rdkit-pypi rdkit-to-params')

import site
site.main()

In [ ]:
#@title Start PyRosetta
import pyrosetta
import pyrosetta_help as ph

#@markdown Do not optimise hydrogen on loading:
no_optH = False #@param {type:"boolean"}
#@markdown Ignore (True) or raise error (False) if novel residue (e.g. ligand) —  **don't tick this**.
ignore_unrecognized_res=False  #@param {type:"boolean"}
#@markdown Use autogenerated PDB residues are often weird (bad geometry, wrong match, protonated etc.): —best do it properly and parameterise it, so **don't tick this**.
load_PDB_components=False  #@param {type:"boolean"}
#@markdown Ignore all waters:
ignore_waters=False  #@param {type:"boolean"}

extra_options= ph.make_option_string(no_optH=no_optH,
                                  ex1=None,
                                  ex2=None,
                                  mute='all',
                                  ignore_unrecognized_res=ignore_unrecognized_res,
                                  load_PDB_components=load_PDB_components,
                                  ignore_waters=ignore_waters)


# capture to log
logger = ph.configure_logger()
pyrosetta.init(extra_options=extra_options)

In [ ]:
#@title Upload acceptor model
#@markdown Upload PDB file of the model into which the ligand(s) will be added.
#@markdown this will be used in a first instance to extract the sequence for blasting.
from google.colab import files
#@markdown Alternatively, use the EBI-AlphaFold2 by providing a uniprot id
#@markdown (leave blank for upload).
uniprot= ''  #@param {type:"string"}

#@markdown If the model has novel ligands, they will be loaded.
#@markdown But to do this a residue type  (=topology) needs to be made or loaded.
#@markdown These are saved as "params files".
#@markdown These following options control both the "acceptor" and "donor" poses (if uploaded).

#@markdown ## Params

#@markdown * Some compounds are parameterised in the database folder of rosetta,
#@markdown others in the PDB component database (if loaded).
#@markdown * Uses the params defined in the cell of the acceptor pose.
#@markdown * If there is no topology avalaible one will be made.
#@markdown * If a params file is present in the working folder it will use it.
#@markdown * See below or visit https://params.mutanalyst.com/ to generate them (upload the with the folder icon on the left).

#@markdown This forces it (a bit silly):
force_parameterisation = False  #@param {type:"boolean"}
#@markdown If it needs to be parameterised make it protonated for pH 7?
neutralise_params=True #@param {type:"boolean"}
save_params=True #@param {type:"boolean"}


#@markdown If a params file is present in the working folder it will use it.
#@markdown Leave this blank... otherwise  (comma separated w/ no rando spaces):
extra_params_files_to_use = '' #@param {type:"string"}
extra_params = [f for f in extra_params_files_to_use.split(',') if f]
use_all_folder_params= ''  #@param {type:"boolean"}
if use_all_folder_params:
  present_params = [filename for filename in os.listdir() if os.path.splitext(filename) == '.params']
else:
  present_params = []
print('loading pose...')
if uniprot:
  acceptor_pose = ph.pose_from_alphafold2(uniprot)
else:
  uploaded = files.upload()
  assert len(uploaded) ==1, 'wrong number of files (only one plz)'
  pdbblock = list(uploaded.values())[0]
  acceptor_pose = ph.ligands.load.parameterised_pose_from_pdbblock(pdbblock,
                                     wanted_ligands = [],
                                     force_parameterisation=force_parameterisation,
                                     neutralise_params=neutralise_params,
                                     save_params=save_params,
                                     overriding_params=extra_params+present_params)

In [ ]:
#@title Find related structures in the PDB
#@markdown Chain of interest (always A in EBI-AF2):
acceptor_chain = 'A' #@param {type:"string"}
#@markdown Download a table of matches?
download_csv = False #@param {type:"boolean"}

print('blasting sequence...')
chain_idx = ph.chain_letter_to_number(acceptor_chain, acceptor_pose)
seq = acceptor_pose.chain_sequence(chain_idx)
hunter = ph.LigandHunter(seq)
print('Most common ligands:')
for lig, c in hunter.get_most_common_ligands()[:10]:
    print(lig, c, hunter.ligand_data[lig][0]['name'])
df = hunter.to_dataframe()
if download_csv:
  df.to_csv('blast_hits.csv')
  files.download('blast_hits.csv')
df

In [ ]:
#@title Choose the closest?

#@markdown Find closest PDB with a given ligand (3-letter code).
#@markdown ` CA` etc. are fine —The front space will be added.
wanted_ligand = '' #@param {type:"string"}
entry = hunter.get_pdb_entry_by_ligand(wanted_ligand)
print(entry)

In [ ]:
#@title load donor pose
#@markdown If the result of the previous cell is good, leave these blank, 
#@markdown else provide a 4-letter PDB code and chain below:
code = '' #@param {type:"string"}
donor_chain = '' #@param {type:"string"}
if not code:
  code = entry['pdb_code']
  donor_chain = entry['chain']

#@markdown If the `cofactor_codes` from the output cell above are not okay,
#@markdown declare which of its ligands you want (comma separated, 
#@markdown don't worry about spaces for ions, e.g. `GTP,MG`):
wanted_ligands = '' #@param {type:"string"}
if not wanted_ligands:
  wanted_ligands = entry['cofactor_codes']
else:
  wanted_ligands = wanted_ligands.replace(' ', '').split(',')

#@markdown If the wanted ligand is not quite the one available (e.g. GDP+Pi or GNP vs. GTP),
#@markdown [Fragmenstein](https://github.com/matteoferla/Fragmenstein) is an option
#@markdown ([example](https://blog.matteoferla.com/2020/07/switching-ligand-in-pdb-with.html))
#@markdown —let me know and I'll add a cell.

#@markdown Parameterisation options taken from previous cell.

# recache
if use_all_folder_params:
  present_params = [filename for filename in os.listdir() if os.path.splitext(filename) == '.params']
else:
  present_params = []

pdb_filename = ph.download_pdb(code)
nicked = ph.LigandNicker(pdb_filename=pdb_filename,
                         chain=donor_chain,
                         neutralise_params=neutralise_params,
                         wanted_ligands=wanted_ligands,
                         force_parameterisation=force_parameterisation,
                         overriding_params=extra_params+present_params)

In [ ]:
#@title Migrate the ligand

#@markdown The steps done in this cell are several. Copypaste of docstring of migrate:
#@markdown > This method aligns the sequences of the acceptor and donor pose.
#@markdown > It finds the mapping of the neighbourhood of the wanted residues of the donor_pose
#@markdown > It superimposes the poses by those residues.
#@markdown > It adds the residues that need nicking.
#@markdown > It adds constraints (optionally) based on the hydrogen bonding of the residues around the wanted residues.
#@markdown > onto the ``.acceptor_pose``. 
#@markdown > It then optionally relaxes the neighbourhood.

#@markdown (In the code the class is called LigandNicker, 
#@markdown which I realised afterwards `to nick` is British English slang
#@markdown whereas most of the rest of code in American English as per custom)

#@markdown leave the following as they are:
constrained=True #@param {type:"boolean"}
relaxed=True #@param {type:"boolean"}
relax_radius=20  #@param {type:"integer"}
relax_cycles=3  #@param {type:"integer"}
nicked.migrate(acceptor_pose, acceptor_chain,
               constrained=constrained,
               relaxed=relaxed,
               relax_radius=relax_radius,
               relax_cycles=relax_cycles)

In [ ]:
#@title Download PDB
nicked.acceptor_pose.dump_pdb('added.pdb')
files.download('added.pdb')

In [ ]:
#@title Upload to Michelanglo (optional)
#@markdown [Michelanglo](https://michelanglo.sgc.ox.ac.uk/) is a website that
#@markdown allows the creation, annotation and sharing of a webpage with an interactive protein viewport.
#@markdown ([examples](https://michelanglo.sgc.ox.ac.uk/gallery)).
#@markdown The created pages are private —they have a 1 in a quintillion change to be guessed within 5 tries.

#@markdown Registered users (optional) can add interactive annotations to pages.
#@markdown A page created by a guest is editable by registered users with the URL to it
#@markdown (this can be altered in the page settings).
#@markdown Leave blank for guest (it will not add an interactive description):

username = ''  #@param {type:"string"}
password = ''  #@param {type:"string"}

import os
assert not os.system(f'pip3 install michelanglo-api')
import site
site.main()
from michelanglo_api import MikeAPI, Prolink
if not username:
  mike = MikeAPI.guest_login()
else:
  mike = MikeAPI(username, password)
    
page = mike.convert_pdb(pdbblock=ph.get_pdbstr(nicked.acceptor_pose),
                        data_selection='ligand',
                        data_focus='residue',
                        )
if username:
     page.retrieve()
     ligands_prolink = Prolink(text='migrated ligand(s)',
                               focus='residue',
                               selection=' or '.join(wanted_ligands))
     
     page.description = '## Description\n\n'
     page.description += f'Model with {ligands_prolink} from {code} (chain {donor_chain}),'
     page.description += 'namely:\n\n'
     for lig in wanted_ligands:
         page.description += '* '+ Prolink(text=lig,
                                          focus='residue',
                                          selection=lig) +'\n'
     page.commit()
page.show_link()